In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from google.colab import drive
from torch.nn import CrossEntropyLoss
import gc

# === CONFIGURAÇÃO OTIMIZADA DE MEMÓRIA ===
def cleanup_memory():
    """Limpa cache da GPU e coleta garbage"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def print_gpu_utilization():
    """Mostra utilização da GPU"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"🖥️ GPU - Alocado: {allocated:.1f}GB | Reservado: {reserved:.1f}GB")

# Configurações globais para economia de memória
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Limpeza inicial
cleanup_memory()

drive.mount('/content/drive')

# === CONFIGURAÇÃO DO MODELO ===
model_id = "Emilio407/guarani-jopara-gemma-2-2b-it-v1"
print(f"🔄 Carregando modelo: {model_id}")

# Carrega tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer carregado")

# Carrega modelo com otimizações
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Usa half precision desde o carregamento
    device_map="auto",
    low_cpu_mem_usage=True,
)
print("✅ Modelo carregado com otimizações")
print_gpu_utilization()

# === PREPARAÇÃO DOS DADOS OTIMIZADA ===
print("\n🔄 Carregando e preparando dados...")
df = pd.read_json("/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json")

# Divide os dados (usando menos dados se necessário para economizar memória)
if len(df) > 1000:  # Se tiver muitos dados, usa uma amostra
    df = df.sample(n=1000, random_state=42)
    print(f"⚠️ Usando amostra de 1000 exemplos para economizar memória")

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"📊 Treino: {len(train_df)} | Validação: {len(val_df)}")

# Converte para Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Função de tokenização otimizada
def tokenize(example):
    prompt = example["instruction"] + "\n" + example["input"] + "\n"
    target = example["output"]
    full_text = prompt + target

    # Tokeniza com max_length menor para economizar memória
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=256,  # Reduzido para economizar memória
        return_tensors=None  # Não retorna tensors para economizar memória
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 Aplicando tokenização...")
tokenized_train = train_dataset.map(
    tokenize,
    remove_columns=train_dataset.column_names,
    batched=False,  # Processa um por vez para economizar memória
    load_from_cache_file=False
)
tokenized_val = val_dataset.map(
    tokenize,
    remove_columns=val_dataset.column_names,
    batched=False,
    load_from_cache_file=False
)

cleanup_memory()
print("✅ Tokenização concluída")

# Data collator otimizado
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt"
)

# === FUNÇÃO DE MÉTRICAS OTIMIZADA ===
def compute_metrics(eval_pred):
    """Função de métricas otimizada para economizar memória"""
    predictions, labels = eval_pred

    # Move para CPU se necessário
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.detach().cpu()
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu()

    # Calcula perplexity de forma mais eficiente
    try:
        shift_logits = predictions[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        shift_logits = shift_logits.view(-1, shift_logits.size(-1))
        shift_labels = shift_labels.view(-1)

        # Remove padding
        mask = shift_labels != -100
        shift_logits = shift_logits[mask]
        shift_labels = shift_labels[mask]

        # Calcula loss
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(shift_logits.float(), shift_labels)
        perplexity = torch.exp(loss)

        return {
            "eval_loss": loss.item(),
            "perplexity": perplexity.item()
        }
    except Exception as e:
        print(f"⚠️ Erro no cálculo de métricas: {e}")
        return {"eval_loss": 0.0, "perplexity": 1000.0}

# === ARGUMENTOS DE TREINAMENTO ULTRA-OTIMIZADOS ===
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado",

    # Configurações de batch otimizadas
    per_device_train_batch_size=1,      # Muito pequeno para economizar memória
    per_device_eval_batch_size=1,       # Muito pequeno para avaliação
    gradient_accumulation_steps=8,       # Simula batch_size=8

    # Configurações de treinamento
    num_train_epochs=2,                  # Reduzido para teste inicial
    learning_rate=2e-5,                  # Learning rate conservador
    warmup_ratio=0.1,                    # Warmup para estabilidade

    # Configurações de avaliação
    eval_strategy="steps",
    eval_steps=100,                      # Avalia menos frequentemente
    eval_accumulation_steps=1,           # Não acumula durante avaliação

    # Configurações de salvamento
    save_strategy="steps",
    save_steps=200,                      # Salva menos frequentemente
    save_total_limit=1,                  # Mantém apenas 1 checkpoint

    # Otimizações de memória CRÍTICAS
    fp16=False,                          # Half precision - ESSENCIAL!
    dataloader_pin_memory=False,        # Economiza memória
    gradient_checkpointing=True,        # Troca computação por memória
    dataloader_num_workers=0,           # Sem workers paralelos
    remove_unused_columns=True,         # Remove colunas desnecessárias

    # Configurações de otimizador
    optim="adamw_torch",                # Otimizador eficiente
    weight_decay=0.01,
    adam_epsilon=1e-8,

    # Logging e relatórios
    logging_dir="logs",
    logging_steps=50,
    logging_first_step=True,

    # Configurações de modelo
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Desabilita relatórios externos
    report_to=[],

    # Configurações adicionais para estabilidade
    max_grad_norm=1.0,                  # Gradient clipping
    prediction_loss_only=False,
)

print("\n🔄 Iniciando treinamento com configurações otimizadas...")
print_gpu_utilization()

# === TRAINER OTIMIZADO ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# === TREINAMENTO COM MONITORAMENTO ===
try:
    print("🚀 Iniciando treinamento...")
    trainer.train()
    print("✅ Treinamento concluído!")

    # Salva o modelo
    output_dir = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"✅ Modelo salvo em: {output_dir}")

except Exception as e:
    print(f"❌ Erro durante treinamento: {e}")
    print("💡 Tentando com configurações ainda mais conservadoras...")

    # Configurações de emergência
    training_args.per_device_train_batch_size = 1
    training_args.gradient_accumulation_steps = 4
    training_args.num_train_epochs = 1

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train.select(range(min(100, len(tokenized_train)))),  # Usa só 100 exemplos
        eval_dataset=tokenized_val.select(range(min(20, len(tokenized_val)))),
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()
    trainer.save_model(output_dir)

# === LIMPEZA ANTES DA AVALIAÇÃO ===
del trainer
cleanup_memory()
print("\n🧹 Memória limpa após treinamento")
print_gpu_utilization()

# === AVALIAÇÃO OTIMIZADA ===
print("\n=== INICIANDO AVALIAÇÃO ===")

# Carrega modelo treinado com otimizações
model_path = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"
tokenizer_trained = AutoTokenizer.from_pretrained(model_path)
model_trained = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)
model_trained.eval()

print("✅ Modelo treinado carregado para avaliação")

# Função de avaliação otimizada
def avaliar_modelo_otimizado(perguntas_teste):
    """Avalia o modelo de forma otimizada"""
    resultados = []

    for i, pergunta in enumerate(perguntas_teste):
        try:
            inputs = tokenizer_trained(pergunta, return_tensors="pt")

            with torch.no_grad():
                outputs = model_trained.generate(
                    inputs['input_ids'],
                    max_length=80,  # Reduzido para economizar
                    do_sample=True,
                    temperature=0.7,
                    num_return_sequences=1,
                    pad_token_id=tokenizer_trained.eos_token_id,
                    early_stopping=True
                )

            resposta = tokenizer_trained.decode(outputs[0], skip_special_tokens=True)
            resposta_limpa = resposta[len(pergunta):].strip()

            resultados.append({
                'pergunta': pergunta,
                'resposta': resposta_limpa
            })

            print(f"Pergunta {i+1}: {pergunta}")
            print(f"Resposta: {resposta_limpa}")
            print("-" * 50)

            # Limpa memória a cada iteração
            del inputs, outputs
            cleanup_memory()

        except Exception as e:
            print(f"⚠️ Erro na pergunta '{pergunta}': {e}")

    return resultados

# Perguntas de teste
perguntas_teste = [
    "Como se diz 'gavião' em Guarani?",
    "Como se diz 'água' em Guarani?",
    "Como se diz 'casa' em Guarani?",
    "Como se diz 'sol' em Guarani?",
    "Como se diz 'lua' em Guarani?"
]

print("🔄 Testando modelo treinado...")
resultados = avaliar_modelo_otimizado(perguntas_teste)

# === CÁLCULO DE PERPLEXITY OTIMIZADO ===
def calcular_perplexity_otimizado(model, tokenizer, dataset, max_samples=30):
    """Calcula perplexity de forma otimizada"""
    model.eval()
    total_loss = 0
    total_tokens = 0
    processed = 0

    # Usa amostra muito pequena para economizar memória
    indices = np.random.choice(len(dataset), min(max_samples, len(dataset)), replace=False)
    sample_dataset = dataset.select(indices)

    print(f"🔄 Calculando perplexity em {len(sample_dataset)} exemplos...")

    with torch.no_grad():
        for example in sample_dataset:
            try:
                input_ids = torch.tensor([example['input_ids']], dtype=torch.long)
                labels = torch.tensor([example['labels']], dtype=torch.long)

                outputs = model(input_ids, labels=labels)
                loss = outputs.loss

                # Conta tokens válidos
                mask = labels != -100
                num_tokens = mask.sum().item()

                if num_tokens > 0:
                    total_loss += loss.item() * num_tokens
                    total_tokens += num_tokens

                processed += 1
                if processed % 10 == 0:
                    print(f"   Processados: {processed}/{len(sample_dataset)}")

                # Limpa memória
                del input_ids, labels, outputs
                cleanup_memory()

            except Exception as e:
                print(f"⚠️ Erro no exemplo {processed}: {e}")
                continue

    if total_tokens > 0:
        avg_loss = total_loss / total_tokens
        perplexity = np.exp(avg_loss)
    else:
        perplexity = float('inf')

    return perplexity

# Calcula perplexity do modelo treinado
print("\n🔄 Calculando métricas do modelo treinado...")
perplexity_trained = calcular_perplexity_otimizado(model_trained, tokenizer_trained, tokenized_val)
print(f"📊 Perplexity modelo treinado: {perplexity_trained:.2f}")

# Remove modelo treinado da memória
del model_trained
cleanup_memory()
print("🧹 Modelo treinado removido da memória")

# === COMPARAÇÃO COM MODELO BASE ===
print("\n🔄 Carregando modelo base para comparação...")
try:
    model_base = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    model_base.eval()

    # Testa modelo base
    print("🔄 Testando modelo base...")
    pergunta_teste = "Como se diz 'gavião' em Guarani?"
    inputs = tokenizer(pergunta_teste, return_tensors="pt")

    with torch.no_grad():
        outputs_base = model_base.generate(
            inputs['input_ids'],
            max_length=80,
            do_sample=False,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta_base = tokenizer.decode(outputs_base[0], skip_special_tokens=True)
    resposta_base_limpa = resposta_base[len(pergunta_teste):].strip()

    # Calcula perplexity do modelo base
    perplexity_base = calcular_perplexity_otimizado(model_base, tokenizer, tokenized_val)

    # Resultados finais
    print(f"\n=== RESULTADOS FINAIS ===")
    print(f"📊 Perplexity modelo base: {perplexity_base:.2f}")
    print(f"📊 Perplexity modelo treinado: {perplexity_trained:.2f}")

    print(f"\n🔍 TESTE COMPARATIVO: '{pergunta_teste}'")
    print(f"Modelo Base: {resposta_base_limpa}")

    # Carrega modelo treinado novamente para comparação
    model_trained_comp = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )

    with torch.no_grad():
        outputs_trained = model_trained_comp.generate(
            inputs['input_ids'],
            max_length=80,
            do_sample=False,
            num_return_sequences=1,
            pad_token_id=tokenizer_trained.eos_token_id
        )

    resposta_trained = tokenizer_trained.decode(outputs_trained[0], skip_special_tokens=True)
    resposta_trained_limpa = resposta_trained[len(pergunta_teste):].strip()
    print(f"Modelo Treinado: {resposta_trained_limpa}")

    if perplexity_trained < perplexity_base:
        melhoria = perplexity_base - perplexity_trained
        print(f"\n✅ SUCESSO! O fine-tuning melhorou o modelo!")
        print(f"🎯 Redução na perplexity: {melhoria:.2f}")
    else:
        diferenca = perplexity_trained - perplexity_base
        print(f"\n⚠️ O modelo ainda precisa de ajustes")
        print(f"📈 Aumento na perplexity: {diferenca:.2f}")
        print("💡 Sugestões: mais dados, mais épocas, ou ajustar learning rate")

except Exception as e:
    print(f"⚠️ Erro na comparação: {e}")
    print("✅ Mas o treinamento foi concluído com sucesso!")

# Limpeza final
cleanup_memory()
print(f"\n🎯 Avaliação concluída! Modelo salvo em: {output_dir}")
print_gpu_utilization()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 Carregando modelo: Emilio407/guarani-jopara-gemma-2-2b-it-v1
✅ Tokenizer carregado


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Modelo carregado com otimizações
🖥️ GPU - Alocado: 14.6GB | Reservado: 15.7GB

🔄 Carregando e preparando dados...
📊 Treino: 224 | Validação: 57
🔄 Aplicando tokenização...


Map:   0%|          | 0/224 [00:00<?, ? examples/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

✅ Tokenização concluída

🔄 Iniciando treinamento com configurações otimizadas...
🖥️ GPU - Alocado: 14.6GB | Reservado: 15.7GB
🚀 Iniciando treinamento...


/tmp/ipython-input-2-5183507.py:210: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


❌ Erro durante treinamento: CUDA out of memory. Tried to allocate 1.10 GiB. GPU 0 has a total capacity of 22.16 GiB of which 1.03 GiB is free. Process 21898 has 21.13 GiB memory in use. Of the allocated memory 20.60 GiB is allocated by PyTorch, and 304.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
💡 Tentando com configurações ainda mais conservadoras...


/tmp/ipython-input-2-5183507.py:241: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.10 GiB. GPU 0 has a total capacity of 22.16 GiB of which 1.03 GiB is free. Process 21898 has 21.13 GiB memory in use. Of the allocated memory 20.60 GiB is allocated by PyTorch, and 304.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)